In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
import bayes

## Example 1: AB test 
#### We have run an email campaign testing 2 different call to actions. As a result, we obtained a click-through rate of 5% for call to action A and 4% for call to action B. Overall, we sent 5,000 emails, 3,500 in variant A and 1,500 in variant B of the call to action.

In [4]:
CI,P = bayes.computeBayesTest([0.2857142857142857,0.25],sample_size=[280,240])
P

,A,B
A,NaN,0.82
B,0.18,NaN


In [5]:
CI,P = bayes.computeBayesTest([0.05,0.04],sample_size=[3500,1500])

#### There is a 94% probability that variant A is better than variant B:

In [6]:
P

,A,B
A,NaN,0.94
B,0.06,NaN


#### The uplift we would get by using variant A over B is between 10% and 41% (with a 80% probability):

In [7]:
CI

,A,B
A,NaN,"[1.1, 1.41]"
B,"[0.71, 0.91]",NaN


## Example 2: AB test with a prior 
#### Similar set up as before, but this time we want to use some previous information. Before this campaign we had run two campaigns, one with call to action A (volume of 700 emails, obtaining a click-through rate of 4%) and another one with call to action B (volume of 300 emails, click-through rate of 4.5%). We can capture this information in the form of prior probabilities through the prior_settings dictionary

In [8]:
CI,P = bayes.computeBayesTest([0.05,0.04],sample_size=[3500,1500],
                              prior_settings={"positive_rate_prior":[0.04,0.045],"sample_size_prior":[700,300]})

#### There is now a 90% probability that variant A is better than variant B. 
#### This is lower than in the previous example because we now have a prior belief (derived from the two previous campaigns) that variant B should actually perform slightly better than A (4.5% vs 4%). The previous campaigns had smaller sample sizes than the latest one (500 and 300, as opposed to 1,000 for the latest campaign) so we still conclude that A is better than B with a 90% probability. 

In [9]:
P

,A,B
A,NaN,0.9
B,0.1,NaN


In [10]:
CI

,A,B
A,NaN,"[1.05, 1.32]"
B,"[0.76, 0.95]",NaN


## Example 3: Multivariate test with 3 variants
#### We have run an email campaign testing 3 different subject lines. As a result, we obtained an open rate of 35% for subject line A, 32% for subject line B and 33% for subject line C. Overall, we sent 6,000 emails, equally split among the three subject lines

In [11]:
CI,P = bayes.computeBayesTest([0.35,0.32,0.33],sample_size=6000)

#### The uplift we would get by using variant A over B is between 5% and 13% (with a 80% probability). The uplift we would get by using variant A over C is between 2% and 10% (with a 80% probability)

In [12]:
CI

,A,B,C
A,NaN,"[1.05, 1.13]","[1.02, 1.1]"
B,"[0.88, 0.95]",NaN,"[0.93, 1.01]"
C,"[0.91, 0.98]","[0.99, 1.08]",NaN


#### There is a ~98% probability that variant A is better than variant B. There is a 75% probability that variant C is better than B.

In [13]:
P

,A,B,C
A,NaN,0.98,0.91
B,0.02,NaN,0.25
C,0.09,0.75,NaN


## Example 4: Sample size calculation for an AB test
#### We are planning an AB test to see if a new recommendation engine would drive a higher cross-sell rate across our customers. The current cross-sell rate using the old engine is 10% and we want to make sure we collect enough volume to detect a relative uplift of at least 20% (i.e. a new cross-sell of 12%)

In [14]:
relative_uplift = 0.2
current_xsell_rate = 0.1
hypotetical_new_xsell_rate = 0.1 + 0.1*relative_uplift

#### The minimum sample size required is 2,237:

In [15]:
bayes.computeBayesTestSampleSize([current_xsell_rate,hypotetical_new_xsell_rate],min_confidence=80.0)

2237

#### If instead we wanted to detect a 6x relative uplift (600%), the function will inform us that the uplift we want to measure is so large that any sample size of at least 10 data points would do

In [16]:
relative_uplift = 6.
current_xsell_rate = 0.1
hypotetical_new_xsell_rate = 0.1 + 0.1*relative_uplift

In [17]:
bayes.computeBayesTestSampleSize([current_xsell_rate,hypotetical_new_xsell_rate])

The values in the explored size range are all large enough to confirm that the positive rates are truly different with a 80.0% Bayesian confidence. The smallest sample size (10.0) corresponds to a Bayesian confidence of 96.9%.


#### If instead we wanted to detect a very small relative uplift (0.01%), the function will inform us that the uplift we want to measure is so small that not even the largest sample size analysed (100,000,000) would be sufficient

In [18]:
relative_uplift = 0.0001
current_xsell_rate = 0.1
hypotetical_new_xsell_rate = 0.1 + 0.1*relative_uplift

In [19]:
bayes.computeBayesTestSampleSize([current_xsell_rate,hypotetical_new_xsell_rate])

The values in the explored size range are too small to confirm that the positive rates are truly different with a 80.0% Bayesian confidence. The largest value in the range (100000000.0) corresponds to a Bayesian confidence of 19.7%. Try smaller Bayesian confidence


### More info from the docstring of the function:

In [20]:
help(bayes.computeBayesTestSampleSize)

Help on function computeBayesTestSampleSize in module bayes:

computeBayesTestSampleSize(true_positive_rates, min_confidence=80.0, accuracy=10, prior_settings={'positive_rate_prior': None, 'sample_size_prior': 0})
    Calculate minimum sample size required to detect a given difference between positive rates.
    
    When designing an AB or Multivariate test, you want to make sure that the sample size you will 
    collect is large enough to confidently conclude that the variations are different when their true 
    positive rates are sufficiently dissimilar.
    
    For an AB test, a way to do this within a Bayesian framework is measuring the required minimum 
    sample size that detects a difference between the two variants with at least a x% Bayesian confidence
    at least p% of the times, given the two expected positive rates (true_positive_rates).
    In this function, we chose p = 80% (fixed) whereas "x" can be selected by the user (min_confidence,
    default value is 80%).
 